In [1]:
import pandas as pd

In [4]:
example = pd.read_csv(
    '/home/yutanagano/UCLOneDrive/MBPhD/data/tcr_embedder/data_sources/tanno_processed/data/A1 memory.txt',
    sep='\t'
)

In [5]:
example = example[
    ['VL', 'CDRL3_AA', 'JL', 'VH', 'CDRH3_AA', 'JH']
]

,Clustered,Collapsed,ClusterID,CDRH3_AA,CDRL3_AA,CDRH3_NT,CDRL3_NT,VH,DH,JH,VL,JL
0,8899,5544,1,CASSLQGARETQYF,CALLSTGGGNKLTF,TGTGCCAGCAGCTTACAGGGGGCGCGGGAGACCCAGTACTTC,TGTGCTCTCCTTTCCACGGGAGGAGGAAACAAACTCACCTTT,TRBV11-2,TRBD1,TRBJ2-5,TRAV9-2,TRAJ10
1,1945,1077,2,CSWKREIVEQFF,CAVSDRNTNAGKSTF,TGCAGTTGGAAGCGGGAGATAGTTGAGCAGTTCTTC,TGTGCTGTGAGTGATAGGAACACCAATGCAGGCAAATCAACCTTT,TRBV20-1,TRBD2,TRBJ2-1,TRAV8-4,TRAJ27
2,1215,867,3,CASSSQGANTEAFF,CIVRKRNQFYF,TGTGCCAGCAGCTCCCAAGGAGCGAACACTGAAGCTTTCTTT,TGCATCGTCAGAAAACGTAACCAGTTCTATTTT,TRBV9,NaN,TRBJ1-1,TRAV26-1,TRAJ49
3,1185,866,4,CASSFQGQGGQPQHF,CAVRPRDTGGFKTIF,TGTGCCAGCAGTTTCCAGGGACAGGGCGGTCAGCCCCAGCATTTT,TGTGCTGTGAGGCCTCGGGATACTGGAGGCTTCAAAACTATCTTT,TRBV12-3,TRBD1,TRBJ1-5,TRAV21,TRAJ9
4,1002,785,5,CSAPLAGVSYNEQFF,CIVRSYNYGQNFVF,TGCAGTGCGCCACTAGCGGGCGTCTCCTACAATGAGCAGTTCTTC,TGCATCGTCAGATCTTATAACTATGGTCAGAATTTTGTCTTT,TRBV20-1,TRBD2,TRBJ2-1,TRAV26-1,TRAJ26
...,...,...,...,...,...,...,...,...,...,...,...,...
28143,2,2,28144,CTSGFRSTGELFF,CAGQSGGGGSNYKLTF,TGTACCAGCGGTTTTAGGAGCACCGGGGAGCTGTTTTTT,TGTGCTGGGCAGTCGGGGGGCGGAGGTAGCAACTATAAACTGACATTT,TRBV2,NaN,TRBJ2-2,TRAV35,TRAJ53
28144,2,2,28145,CTSSETASSYEQYF,CALKRGFGNVLHC,TGTACCAGCAGTGAAACGGCTAGCAGCTACGAGCAGTACTTC,TGTGCTCTGAAAAGGGGCTTTGGGAATGTGCTGCATTGC,TRBV2,TRBD2,TRBJ2-7,TRAV9-2,TRAJ35
28145,2,2,28146,CTSSREQGSTDTQYF,CAVELVRADGLTF,TGCACCAGCAGCAGAGAACAGGGGTCCACAGATACGCAGTATTTT,TGTGCCGTGGAATTGGTTCGGGCTGACGGACTCACCTTT,TRBV5-1,TRBD1,TRBJ2-3,TRAV39,TRAJ45
28146,2,2,28147,CTSSVRTGGPSSYGYTF,CIAYGSSNTGKLIF,TGTACCAGCAGCGTTAGGACAGGGGGCCCCTCTAGCTATGGCTACA...,TGCATCGCTTACGGCTCTAGCAACACAGGCAAACTAATCTTT,TRBV9,TRBD1,TRBJ1-2,TRAV26-1,TRAJ37
